In [ ]:
import glob

path = '/Users/kris/data_sources/fdic/BulkReports/'

In [ ]:
for folder in glob.glob(path + '*[!.]'):
    print(f'{folder}')

In [ ]:
folders = glob.glob(path + '*[!.]')
print(f'{folders}')

In [ ]:
import pandas as pd
import datetime

df = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])
df0 = pd.DataFrame(columns=['Date', 'As_of'])
df1 = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])

#df = pd.concat([df, df1], ignore_index=True)

In [ ]:
df0 = pd.concat([df0, df1], ignore_index=True)

In [ ]:
df = pd.concat(df1, df1)

In [ ]:
#type(df['Date'])
#df['Date'].isin(['20240506'])

'20240506' in df['Date'].values

In [ ]:
df.shape[0]

In [ ]:
skipped_dtypes = {'Date': str, 'As_of': datetime.datetime}
skipped_dtypes.keys()

In [ ]:
import time

t = time.time()
period = 60

diff = t - period

current_time = time.strftime('%H:%M:%S', time.localtime(t))
wait_time = time.strftime('%H:%M:%S', time.localtime(diff))

milliseconds_current = int((t - int(t)) * 1000)
milliseconds_diff = int((diff - int(diff)) * 1000)

print(f'current: {current_time}.{milliseconds_current}\nwait: {wait_time}.{milliseconds_diff}\n')


In [ ]:
import time
import datetime

current = time.time()
#print(datetime.datetime.fromtimestamp(current + 3600).strftime("%Y%m%d %H:%M:%S"))
print(f'{time.strftime("%Y%m%d %H:%M:%S",time.gmtime(current + 3600))}')

In [ ]:
#ij = 0
for i in range(10):
    for j in range(5):
        ij +=1
print(ij)

In [ ]:
# Paths to your XBRL file and schema file

bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_base_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 09302023_Form051'
#schema_base_path = '/Users/kris/Downloads/_ 09302023_Form051/'

xbrl_file_path = bulkReport_path + 'ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
schema_file_path = schema_base_path + 'call-report051-2023-09-30-v261.xsd'
presentation_file_path = schema_base_path + 'call-report051-2023-09-30-v261-pres.xml'

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = '/Users/kris/data_sources/fdic/BulkReports/ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
#schema_file_path = '/Users/kris/Downloads/_ 09302023_Form051/call-report051-2023-09-30-v261.xsd'

def parse_xsd(schema_path):
    with open(schema_path, 'rb') as schema_file:
        schema_root = etree.XML(schema_file.read())
    return schema_root

def parse_xbrl(xbrl_path):
    with open(xbrl_path, 'rb') as xbrl_file:
        xbrl_root = etree.XML(xbrl_file.read())
    return xbrl_root

def extract_data_from_xbrl(xbrl_root, schema_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path):
    schema_root = parse_xsd(schema_file_path)
    xbrl_root = parse_xbrl(xbrl_file_path)
    data = extract_data_from_xbrl(xbrl_root, schema_root)
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Optionally, parse and use the presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    
    # Present data
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    arcrole = 'http://www.xbrl.org/2003/arcrole/parent-child'
    role_refs = presentation_root.xpath('//link:roleRef', namespaces=link_namespace)
    
    presentations = {}
    for role_ref in role_refs:
        role_uri = role_ref.get('roleURI')
        locators = presentation_root.xpath(f'//link:loc[@xlink:role="{role_uri}"]', namespaces=link_namespace)
        arcs = presentation_root.xpath(f'//link:definitionArc[@xlink:arcrole="{arcrole}"]', namespaces=link_namespace)
        
        elements = []
        for arc in arcs:
            from_locator = arc.get('from')
            to_locator = arc.get('to')
            from_element = next((loc for loc in locators if loc.get('label') == from_locator), None)
            to_element = next((loc for loc in locators if loc.get('label') == to_locator), None)
            if from_element is not None and to_element is not None:
                elements.append((from_element.get('{http://www.w3.org/1999/xlink}href'), to_element.get('{http://www.w3.org/1999/xlink}href')))
        presentations[role_uri] = elements
    
    return presentations

def present_data(data, presentations):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    presentation_order = []
    for role_uri, elements in presentations.items():
        for parent, child in elements:
            presentation_order.append(child.split('#')[-1])
    
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentations = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentations)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
    
    return presentation_order

def present_data(data, presentation_order):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "/path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "/path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "/mnt/data/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    print("Validating XBRL file")
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    print("Parsing presentation linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    hierarchy = {}
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
            if locators[from_label] not in hierarchy:
                hierarchy[locators[from_label]] = []
            hierarchy[locators[from_label]].append(locators[to_label])
    
    return presentation_order, hierarchy

def present_data(data, presentation_order, hierarchy):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    print("Starting main process")
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order, hierarchy = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order, hierarchy)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20231231.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    return calculations

# Function to parse the definition linkbase with additional debug statements
def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    try:
        root = etree.parse(definition_file)
    except Exception as e:
        print(f"Failed to parse definition linkbase file: {definition_file}. Error: {e}")
        return {}
    
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions
    
def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations, definitions):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display calculation relationships
    print("\nCalculations:")
    for parent, children in calculations.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display definition relationships
    print("\nDefinitions:")
    for parent, children in definitions.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print("\nData:")
    print(df)

def main(xbrl_file_path, schema_files, linkbase_files):
    print("Starting main process")
    schema_roots = [parse_xml(file_path) for file_path in schema_files]
    xbrl_root = parse_xml(xbrl_file_path)
    
    #validate_xbrl(xbrl_root, schema_roots)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse linkbases
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    definitions = parse_definition_linkbase(linkbase_files['definition'])
    
    # Parse additional linkbases if needed
    parse_additional_linkbases(linkbase_files['ec-mess'])
    parse_additional_linkbases(linkbase_files['ec'])
    
    # Present data
    present_data(data, presentation_order, hierarchy, calculations, definitions)

if __name__ == "__main__":
    main(xbrl_file_path, schema_files, linkbase_files)

In [2]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase: {linkbase_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        #print(f"Processing {arc_tag}Arc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Arcs found in {arc_tag} linkbase: {arcs}")
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    print(f"Parsing calculation linkbase: {calculation_file}")
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    print(f"Calculations: {calculations}")
    return calculations

def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(definition_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        print(f"Processing defArc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions

def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations=None, definitions=None):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}: {children}")
    
    # Display calculations
    if calculations:
        print("\nCalculations:")
        for parent, children in calculations.items():
            print(f"{parent}: {children}")
    
    # Display definitions
    if definitions:
        print("\nDefinitions:")
        for parent, children in definitions.items():
            print(f"{parent}: {children}")

def main():
    # Parsing the provided definition linkbase
    definitions = parse_definition_linkbase(linkbase_files['definition'])

    # Parsing other necessary linkbases
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])

    # Parsing and validating XBRL data
    xbrl_root = parse_xml(xbrl_file_path)
    schema_roots = [parse_xml(schema_file) for schema_file in schema_files]
    #validate_xbrl(xbrl_root, schema_roots)

    # Extracting data from XBRL
    data = extract_data_from_xbrl(xbrl_root)

    # Presenting data
    present_data(data, presentation_order, hierarchy, calculations=calculations, definitions=definitions)

if __name__ == "__main__":
    main()

Parsing definition linkbase: /Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/call-report051-2023-12-31-v265-def.xml
Parsing XML file: /Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/call-report051-2023-12-31-v265-def.xml
Locators found: {'ffiec051_pres-report': 'ffiec051_pres-report', 'cc_RCON9999': 'cc_RCON9999', 'cc_RSSD9050': 'cc_RSSD9050', 'cc_RSSD9017': 'cc_RSSD9017', 'cc_RSSD9130': 'cc_RSSD9130', 'cc_RSSD9200': 'cc_RSSD9200', 'cc_RSSD9220': 'cc_RSSD9220', 'cc_RCON9224': 'cc_RCON9224', 'cc_TEXTC490': 'cc_TEXTC490', 'cc_TEXTC491': 'cc_TEXTC491', 'cc_TEXTC492': 'cc_TEXTC492', 'cc_TEXTC493': 'cc_TEXTC493', 'cc_TEXTC494': 'cc_TEXTC494', 'cc_TEXTC495': 'cc_TEXTC495', 'cc_TEXTC496': 'cc_TEXTC496', 'cc_TEXT4086': 'cc_TEXT4086', 'cc_TEXT8902': 'cc_TEXT8902', 'cc_TEXT9116': 'cc_TEXT9116', 'cc_TEXTB962': 'cc_TEXTB962', 'cc_TEXTB926': 'cc_TEXTB926', 'cc_TEXTB963': 'cc_TEXTB963', 'cc_TEXTB964': 'cc_TEXTB964', 'cc_TEXTC366': 'cc_TEXTC366', 'cc_TEXTC367': '